In [59]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

df  = pd.read_csv("creditcard.csv")[:80_000]
df.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


In [60]:
X = df.drop(columns=['Time','Amount','Class']).values#here we only takes v's that's why we're removing temporarily time ,amount,class columns that is the reason we didn't use inplace =True
y= df['Class'].values
#to emphasize what kind of data set it is we print some useful information
f"shapes of X={X.shape} y={y.shape}, #Fraud Cases={y.sum()}"#x.shape mean it tells that no.of elements in each dimension 

'shapes of X=(80000, 28) y=(80000,), #Fraud Cases=196'

In [61]:
from sklearn.linear_model import LogisticRegression

mod = LogisticRegression(max_iter=1000)#max_iter mean maximum iteration
#here class_weight is a dictionary that allows me to specify how much weight to
#assign to each class and in particular the way you should read it is that for
#class '0' that will be non-frad cases then we assign a wt of 1 but for my class
#'1' that will be frad cases and let's double the wt 2 the idea being that we're 
#gonna get more fraud cases selected.
mod.fit(X,y).predict(X)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [62]:
mod.fit(X,y).predict(X).sum()#if i'm overfitting on the trainset then the model
#detects fewer cases than i actually have in my dataset (fraud cases)

151

In [63]:
mod = LogisticRegression(class_weight={0: 1,1: 2},max_iter=1000)#max_iter mean maximum iteration
#here class_weight is a dictionary that allows me to specify how much weight to
#assign to each class and in particular the way you should read it is that for
#class '0' that will be non-frad cases then we assign a wt of 1 but for my class
#'1' that will be frad cases and let's double the wt 2 the idea being that we're 
#gonna get more fraud cases selected.
mod.fit(X,y).predict(X)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [64]:
mod.fit(X,y).predict(X).sum()#in previous it shows 151 now it shows 171 fraud
#cases bcoz we add the setting class_weight in logistisregression.

171

In [65]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(
    estimator=LogisticRegression(max_iter=1000),
    param_grid={'class_weight':[{0: 1,1: v} for v in range(1,4)]},
    cv=4,
    n_jobs=-1#bcoz gridsearch will occur in parallel
)

grid.fit(X, y)

GridSearchCV(cv=4, estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid={'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 2},
                                          {0: 1, 1: 3}]})

In [66]:
grid.cv_results_#this contains all the results from the cross validation

{'mean_fit_time': array([1.2925936 , 1.08603686, 1.02544749]),
 'std_fit_time': array([0.29849205, 0.28045829, 0.14055181]),
 'mean_score_time': array([0.00459886, 0.00399989, 0.00300217]),
 'std_score_time': array([6.15095145e-04, 4.25662305e-07, 9.97668534e-04]),
 'param_class_weight': masked_array(data=[{0: 1, 1: 1}, {0: 1, 1: 2}, {0: 1, 1: 3}],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'class_weight': {0: 1, 1: 1}},
  {'class_weight': {0: 1, 1: 2}},
  {'class_weight': {0: 1, 1: 3}}],
 'split0_test_score': array([0.99405, 0.99025, 0.9873 ]),
 'split1_test_score': array([0.99835, 0.9984 , 0.99845]),
 'split2_test_score': array([0.99945, 0.9996 , 0.9996 ]),
 'split3_test_score': array([0.9978 , 0.99805, 0.99815]),
 'mean_test_score': array([0.9974125, 0.996575 , 0.995875 ]),
 'std_test_score': array([0.0020302 , 0.0036967 , 0.00498027]),
 'rank_test_score': array([1, 2, 3])}

In [67]:
pd.DataFrame(grid.cv_results_)#this model is predicting no fraud most of the time 
#we're getiing a really high accuracy but this is not the metrics we're intrested in 
#

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,1.292594,0.298492,0.004599,6.150951e-04,"{0: 1, 1: 1}","{'class_weight': {0: 1, 1: 1}}",0.99405,0.99835,0.99945,0.99780,0.997413,0.002030,1
1,1.086037,0.280458,0.004000,4.256623e-07,"{0: 1, 1: 2}","{'class_weight': {0: 1, 1: 2}}",0.99025,0.99840,0.99960,0.99805,0.996575,0.003697,2
2,1.025447,0.140552,0.003002,9.976685e-04,"{0: 1, 1: 3}","{'class_weight': {0: 1, 1: 3}}",0.98730,0.99845,0.99960,0.99815,0.995875,0.004980,3


In [68]:
from sklearn.metrics import precision_score, recall_score

precision_score(y, grid.predict(X))#the way precisionscore work is You can pass the true values 
#those are the values that should predict and you can give it the predicted values 
#predictionscore is like "given that i predict fraud how accurate am i".

0.7682119205298014

In [69]:
recall_score(y, grid.predict(X))#recallscore tell me "did i get all the fraud cases" it has to
#recall whether it got or not.
#like if you take an example: if i were to say hey let's predict that every 
#single case is a fraud case well then the recall score is going to be really
#high and precisionscore is going to be really low.

#Suppose in another example: if i find one candidate that's a fradulent 
#candidate but nobody else get predicted as fraud then i'll have really high 
#precisionscore and low recallscore

0.5918367346938775

In [70]:
#now we're going to add this two metric functions precisionscore and recallscore
#in gridsearch now if you want to use this two funcs in gridsearch then we have 
#to pass that to make_scorer function first
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, make_scorer
grid = GridSearchCV(
    estimator=LogisticRegression(max_iter=1000),
    param_grid={'class_weight':[{0: 1,1: v} for v in np.linspace(1, 20, 30)]},
    scoring={'precision':make_scorer(precision_score), 'recall_score':make_scorer(recall_score)},
    refit='precision',
    return_train_score=True,
    cv=10,
    n_jobs=-1#bcoz gridsearch will occur in parallel  
)

grid.fit(X, y)

GridSearchCV(cv=10, estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid={'class_weight': [{0: 1, 1: 1.0},
                                          {0: 1, 1: 1.6551724137931034},
                                          {0: 1, 1: 2.310344827586207},
                                          {0: 1, 1: 2.9655172413793105},
                                          {0: 1, 1: 3.6206896551724137},
                                          {0: 1, 1: 4.275862068965517},
                                          {0: 1, 1: 4.931034482758621},
                                          {0: 1, 1: 5.586206896551724},
                                          {0: 1, 1: 6.241379310344827},
                                          {0: 1, 1: 6.896551724137931},...
                                          {0: 1, 1: 14.758620689655173},
                                          {0: 1, 1: 15.413793103448276},
                                          {0: 1, 1: 16.06896551724

In [71]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,params,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,...,split2_train_recall_score,split3_train_recall_score,split4_train_recall_score,split5_train_recall_score,split6_train_recall_score,split7_train_recall_score,split8_train_recall_score,split9_train_recall_score,mean_train_recall_score,std_train_recall_score
0,1.515578,0.197725,0.008503,0.002114,"{0: 1, 1: 1.0}","{'class_weight': {0: 1, 1: 1.0}}",1.000000,0.463415,0.583333,1.000000,...,0.627119,0.548023,0.573864,0.573864,0.562500,0.613636,0.636364,0.607955,0.612185,0.054733
1,1.940834,0.283289,0.013347,0.009550,"{0: 1, 1: 1.6551724137931034}","{'class_weight': {0: 1, 1: 1.6551724137931034}}",1.000000,0.463415,0.583333,1.000000,...,0.683616,0.627119,0.670455,0.647727,0.630682,0.687500,0.698864,0.687500,0.680239,0.050286
2,2.084234,0.256070,0.011927,0.009184,"{0: 1, 1: 2.310344827586207}","{'class_weight': {0: 1, 1: 2.310344827586207}}",1.000000,0.463415,0.583333,1.000000,...,0.740113,0.683616,0.710227,0.698864,0.687500,0.715909,0.744318,0.727273,0.724454,0.043881
3,1.789105,0.142563,0.011855,0.005625,"{0: 1, 1: 2.9655172413793105}","{'class_weight': {0: 1, 1: 2.9655172413793105}}",1.000000,0.452381,0.583333,1.000000,...,0.785311,0.706215,0.744318,0.732955,0.715909,0.755682,0.772727,0.738636,0.749978,0.039589
4,1.757474,0.361418,0.011400,0.006621,"{0: 1, 1: 3.6206896551724137}","{'class_weight': {0: 1, 1: 3.6206896551724137}}",1.000000,0.452381,0.583333,1.000000,...,0.824859,0.740113,0.755682,0.744318,0.727273,0.778409,0.784091,0.761364,0.771498,0.037959
5,1.713620,0.191403,0.011201,0.003220,"{0: 1, 1: 4.275862068965517}","{'class_weight': {0: 1, 1: 4.275862068965517}}",1.000000,0.452381,0.583333,1.000000,...,0.841808,0.768362,0.778409,0.789773,0.772727,0.789773,0.795455,0.772727,0.792485,0.029289
6,1.585626,0.169234,0.010232,0.005738,"{0: 1, 1: 4.931034482758621}","{'class_weight': {0: 1, 1: 4.931034482758621}}",1.000000,0.452381,0.583333,1.000000,...,0.847458,0.802260,0.789773,0.818182,0.801136,0.801136,0.818182,0.789773,0.812327,0.021063
7,1.503615,0.257019,0.010680,0.003852,"{0: 1, 1: 5.586206896551724}","{'class_weight': {0: 1, 1: 5.586206896551724}}",1.000000,0.452381,0.583333,1.000000,...,0.847458,0.813559,0.829545,0.829545,0.812500,0.806818,0.835227,0.818182,0.827080,0.017190
8,1.542564,0.167763,0.008983,0.001765,"{0: 1, 1: 6.241379310344827}","{'class_weight': {0: 1, 1: 6.241379310344827}}",0.944444,0.452381,0.583333,0.947368,...,0.847458,0.824859,0.846591,0.835227,0.823864,0.829545,0.857955,0.829545,0.838431,0.013974
9,2.094214,0.669953,0.009155,0.001404,"{0: 1, 1: 6.896551724137931}","{'class_weight': {0: 1, 1: 6.896551724137931}}",0.944444,0.452381,0.583333,0.947368,...,0.853107,0.830508,0.846591,0.835227,0.829545,0.835227,0.863636,0.846591,0.844665,0.012015


In [72]:
plt.figure(figsize=(12, 4))
df_results = pd.DataFrame(grid.cv_results_)
for score in ['mean_test_recall', 'mean_test_precision', 'mean_test_min_both']:
    plt.plot([_[1] for _ in df_results['param_class_weight']], 
             df_results[score], 
             label=score)
plt.legend();

KeyError: 'mean_test_recall'

<Figure size 1200x400 with 0 Axes>